In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("tariff.csv")

In [3]:
df.head()

,PCT_CODE,DESCRIPTION,CD
0,NaN,NaN,NaN
1,101,"Live horses, asses, mules and hinnies.",NaN
2,NaN,- Horses:,NaN
3,10121,- - Pure-bred breeding animals,3
4,10129,- - Other,3


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11967 entries, 0 to 11966
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PCT_CODE     8643 non-null   object
 1   DESCRIPTION  11929 non-null  object
 2   CD           7588 non-null   object
dtypes: object(3)
memory usage: 280.6+ KB


In [5]:
print(df.isna().sum())

PCT_CODE       3324
DESCRIPTION      38
CD             4379
dtype: int64


In [6]:
category_df = df[df['CD'].isna() & df['PCT_CODE'].notna()].reset_index(drop=True)

print("Extracted Categories:")
print(category_df)

Extracted Categories:
     PCT_CODE                                        DESCRIPTION   CD
0         101             Live horses, asses, mules and hinnies.  NaN
1         102                               Live bovine animals.  NaN
2         103                                        Live swine.  NaN
3         104                              Live sheep and goats.  NaN
4         105  Live poultry, that is to say, fowls of the spe...  NaN
...       ...                                                ...  ...
1050     9701  Paintings, drawings and pastels, executed enti...  NaN
1051     9702       Original engravings, prints and lithographs.  NaN
1052     9703  Original sculptures and statuary, in any mater...  NaN
1053     9705  Collections and collectors’ pieces of archaeol...  NaN
1054     9706            Antiques of an age exceeding 100 years.  NaN

[1055 rows x 3 columns]


In [7]:
df_cleaned = df.drop(category_df.index).reset_index(drop=True)

In [8]:
df_cleaned.head()

,PCT_CODE,DESCRIPTION,CD
0,15093,- Virgin olive oil,Rs.5 000/ MT
1,15094,- Other virgin olive oils,Rs.500 0/MT
2,15099,- Other,Rs.6 MT
3,151,"Other oils and their fractions, obtained solel...",NaN
4,15101,- Crude olive pomace oil,Rs.6000/\nMT


In [9]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10912 entries, 0 to 10911
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PCT_CODE     7747 non-null   object
 1   DESCRIPTION  10879 non-null  object
 2   CD           6810 non-null   object
dtypes: object(3)
memory usage: 255.9+ KB


In [10]:
category_df = category_df.drop(columns=['CD'])

In [11]:
print("Extracted Categories:")
print(category_df)

Extracted Categories:
     PCT_CODE                                        DESCRIPTION
0         101             Live horses, asses, mules and hinnies.
1         102                               Live bovine animals.
2         103                                        Live swine.
3         104                              Live sheep and goats.
4         105  Live poultry, that is to say, fowls of the spe...
...       ...                                                ...
1050     9701  Paintings, drawings and pastels, executed enti...
1051     9702       Original engravings, prints and lithographs.
1052     9703  Original sculptures and statuary, in any mater...
1053     9705  Collections and collectors’ pieces of archaeol...
1054     9706            Antiques of an age exceeding 100 years.

[1055 rows x 2 columns]


In [12]:
print(df_cleaned.isna().sum())

PCT_CODE       3165
DESCRIPTION      33
CD             4102
dtype: int64


In [13]:
df_cleaned = df_cleaned.dropna()

In [14]:
df_cleaned.head()

,PCT_CODE,DESCRIPTION,CD
0,15093,- Virgin olive oil,Rs.5 000/ MT
1,15094,- Other virgin olive oils,Rs.500 0/MT
2,15099,- Other,Rs.6 MT
4,15101,- Crude olive pomace oil,Rs.6000/\nMT
5,15109,- Other,Rs.6 000/ MT


In [15]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6810 entries, 0 to 10911
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PCT_CODE     6810 non-null   object
 1   DESCRIPTION  6810 non-null   object
 2   CD           6810 non-null   object
dtypes: object(3)
memory usage: 212.8+ KB


In [16]:
df_cleaned

,PCT_CODE,DESCRIPTION,CD
0,15093,- Virgin olive oil,Rs.5 000/ MT
1,15094,- Other virgin olive oils,Rs.500 0/MT
2,15099,- Other,Rs.6 MT
4,15101,- Crude olive pomace oil,Rs.6000/\nMT
5,15109,- Other,Rs.6 000/ MT
...,...,...,...
10905,970529,- - Other,3
10907,970531,- - Of an age exceeding 100 years,3
10908,970539,- - Other,3
10910,97061,- Of an age exceeding 250 years,3


In [17]:
df_cleaned.to_csv("tariff_cleaned.csv", index=False)
category_df.to_csv("categories_only.csv", index=False)

In [18]:
print(category_df.isna().sum())

PCT_CODE        0
DESCRIPTION    37
dtype: int64


In [19]:
category_df = category_df.dropna()

In [20]:
category_df.to_csv("categories_only.csv", index=False)

In [21]:
mapped_rows = []
current_category = None
current_pct_prefix = None

for idx, row in df.iterrows():
    pct = row['PCT_CODE']
    desc = row['DESCRIPTION']
    cd = row['CD']
    
    if pd.notna(pct) and pd.isna(cd):
        current_category = desc.strip() if isinstance(desc, str) else None
        current_pct_prefix = str(pct)
    elif pd.notna(cd):
        mapped_rows.append((row['PCT_CODE'], row['DESCRIPTION'], row['CD'], current_category, current_pct_prefix))

# Step 2: Build DataFrame with mapped categories
mapped_df = pd.DataFrame(mapped_rows, columns=['PCT_CODE', 'DESCRIPTION', 'CD', 'CATEGORY', 'CATEGORY_PCT'])

# Step 3: Save
mapped_df.to_csv("products_with_categories.csv", index=False)